In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
file_path = "/fs/ess/PAS2136/Rayeed/BritishCarabids/British Carabids/BritishCarabids.csv"
brit_df = pd.read_csv(file_path)
brit_df.rename(columns={"CanonicalName": "ScientificName"}, inplace=True)
cols = ['ImageFilePath', 'ScientificName']
df = brit_df[cols]
df = df.dropna()
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Asaphidion curtum
1,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Asaphidion curtum


In [4]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")

tokenizer = open_clip.get_tokenizer("ViT-B-32")

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

for param in model.parameters() :
    param.requires_grad = False


In [5]:
def extract_clip_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess_val(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_tensor)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    return image_features.cpu().numpy()
    

In [6]:
X = np.vstack([extract_clip_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing Images ...") ])

print(f"Extracted feature shape: {X.shape}")

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])


df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)



Processing Images ...:   0%|          | 0/63077 [00:00<?, ?it/s]

Extracted feature shape: (63077, 512)


In [7]:

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in tqdm(models.items(), desc="Training Models", leave=True):
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 35.32% | Prec: 42.81% | Rec: 35.32% | F1: 35.33% | Bal Acc: 39.55% | MCC: 0.3511
LogisticRegression        | Acc: 71.58% | Prec: 72.21% | Rec: 71.58% | F1: 71.58% | Bal Acc: 69.06% | MCC: 0.7143
NearestNeighbor           | Acc: 48.15% | Prec: 50.90% | Rec: 48.15% | F1: 47.72% | Bal Acc: 42.75% | MCC: 0.4787
MLP_Baseline              | Acc: 70.14% | Prec: 70.67% | Rec: 70.14% | F1: 70.09% | Bal Acc: 67.29% | MCC: 0.6998


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
6930,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Harpalus rubripes,Calathus rotundicollis,Harpalus rubripes,Calathus rotundicollis,Harpalus rubripes
2549,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Bembidion fumigatum,Trechus obtusus,Bembidion fumigatum,Bembidion fumigatum,Bembidion bruxellense


In [9]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BritishCarabids/2.CLIP-BritishCarabids-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BritishCarabids/2.CLIP-BritishCarabids-species-metrics.csv", index=False)